In [ ]:
# Download data

In [159]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta

def DataCleaner(prv) :
    province = prv
    province_pm = prv
    if province.lower() == 'chiangmai' :
        province = 'Chiang Mai'
        province_pm = 'Chiang_Mai'
    elif province.lower() == 'khonkaen' :
        province = 'Khon Kaen'
        province_pm = 'Khon_Kaen'

    # temperature
    df_temp = pd.read_csv('PM2.5_kaggle 2021/'+province+'/Train/3H_temperature_'+province+'.csv')
    df_temp['datetime'] = pd.to_datetime(df_temp['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
    df_temp['lat'] = np.round(df_temp['lat'],4)
    df_temp['long'] = np.round(df_temp['long'],4)

    # wind
    df_wind = pd.read_csv('PM2.5_kaggle 2021/'+province+'/Train/3H_wind_'+province+'.csv')
    df_wind['datetime'] = pd.to_datetime(df_wind['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
    df_wind['lat'] = np.round(df_wind['lat'],4)
    df_wind['long'] = np.round(df_wind['long'],4)

    # PM2.5
    pm = open('PM2.5_kaggle 2021/'+province+'/Train/' + province_pm + '.txt','r')
    df_pm = []
    for line in pm :
        if line[0] == '%' :
            continue
        else :
            row = line.split()
            hour = int(row[3])
            format_time = row[0] + "-" + row[1] + "-" + row[2] + " " + str(hour) + ":00:00"
            format_time = pd.to_datetime(format_time, format='%Y-%m-%d %H:%M:%S', errors='ignore')
            format_time = format_time + timedelta(hours=7)
            format_row = [format_time,float(row[4])]
            df_pm.append(format_row)
    pm.close()
    df_pm = pd.DataFrame(df_pm,columns=['datetime','PM2.5'])
    df_pm['datetime'] = df_pm['datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

    # merge
    df = pd.merge(df_temp, df_wind,  how='left', left_on=['datetime','lat','long'], right_on = ['datetime','lat','long'])
    df['datetime'] = df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    df = df.set_index('datetime').join(df_pm.set_index('datetime'), on='datetime', how='outer', sort=True)
    df.reset_index(drop=True, inplace=True)
    
    # clean Na
    df['Temp(C)'] = df['Temp(C)'].interpolate('linear')
    df['WindDir'] = df['WindDir'].interpolate('linear')
    df['Wind Speed(km/h)'] = df['Wind Speed(km/h)'].interpolate('linear')
    df['PM2.5'] = df['PM2.5'].interpolate('linear')
    df['lat'] = df['lat'].fillna(df['lat'].mean())
    df['long'] = df['long'].fillna(df['long'].mean())
    return df


In [150]:
df_bkk = DataCleaner('Bangkok').iloc[5:,:]
df_bkk

In [ ]:
df_bkk.to_csv('bangkok.csv', index=False)

In [154]:
df_chan = DataCleaner('Chanthaburi').iloc[5:,:]
df_chan

,datetime,Temp(C),lat,long,WindDir,Wind Speed(km/h),PM2.5
5,2016-03-03 15:00:00,32.55,12.6096,102.1045,60.000000,10.500000,44.2
6,2016-03-03 16:00:00,31.70,12.6096,102.1045,55.000000,10.000000,44.2
7,2016-03-03 17:00:00,30.20,12.6096,102.1045,58.333333,12.333333,44.3
8,2016-03-03 18:00:00,28.70,12.6096,102.1045,61.666667,14.666667,44.2
9,2016-03-03 19:00:00,27.20,12.6096,102.1045,65.000000,17.000000,34.2
...,...,...,...,...,...,...,...
25136,2019-03-18 02:00:00,27.50,12.6096,102.1045,65.000000,28.000000,31.9
25137,2019-03-18 03:00:00,27.50,12.6096,102.1045,65.000000,28.000000,30.2
25138,2019-03-18 04:00:00,27.50,12.6096,102.1045,65.000000,28.000000,32.8
25139,2019-03-18 05:00:00,27.50,12.6096,102.1045,65.000000,28.000000,32.0


In [156]:
df_chan.to_csv('chanthaburi.csv', index=False)

In [163]:
df_cm = DataCleaner('ChiangMai').iloc[5:,:]
df_cm

,datetime,Temp(C),lat,long,WindDir,Wind Speed(km/h),PM2.5
5,2016-03-03 15:00:00,33.55,18.7904,98.9847,235.000000,4.000000,53.7
6,2016-03-03 16:00:00,33.40,18.7904,98.9847,230.000000,5.000000,53.7
7,2016-03-03 17:00:00,30.40,18.7904,98.9847,226.666667,6.666667,53.7
8,2016-03-03 18:00:00,27.40,18.7904,98.9847,223.333333,8.333333,53.7
9,2016-03-03 19:00:00,24.40,18.7904,98.9847,220.000000,10.000000,33.4
...,...,...,...,...,...,...,...
25136,2019-03-18 02:00:00,28.00,18.7904,98.9847,260.000000,10.000000,75.8
25137,2019-03-18 03:00:00,28.00,18.7904,98.9847,260.000000,10.000000,75.6
25138,2019-03-18 04:00:00,28.00,18.7904,98.9847,260.000000,10.000000,84.6
25139,2019-03-18 05:00:00,28.00,18.7904,98.9847,260.000000,10.000000,104.8


In [167]:
df_cm.to_csv('chiangmai.csv', index=False)

In [170]:
df_kan = DataCleaner('Kanchanaburi').iloc[5:,:]
df_kan

,datetime,Temp(C),lat,long,WindDir,Wind Speed(km/h),PM2.5
5,2016-03-03 15:00:00,35.900000,14.0041,99.5483,82.5,11.5,65.2
6,2016-03-03 16:00:00,36.700000,14.0041,99.5483,85.0,10.0,65.2
7,2016-03-03 17:00:00,35.066667,14.0041,99.5483,90.0,11.0,64.9
8,2016-03-03 18:00:00,33.433333,14.0041,99.5483,95.0,12.0,64.9
9,2016-03-03 19:00:00,31.800000,14.0041,99.5483,100.0,13.0,38.4
...,...,...,...,...,...,...,...
25152,2019-03-18 02:00:00,29.900000,14.0041,99.5483,85.0,10.0,40.5
25153,2019-03-18 03:00:00,29.900000,14.0041,99.5483,85.0,10.0,43.2
25154,2019-03-18 04:00:00,29.900000,14.0041,99.5483,85.0,10.0,46.9
25155,2019-03-18 05:00:00,29.900000,14.0041,99.5483,85.0,10.0,46.3


In [171]:
df_kan.to_csv('kanchanaburi.csv', index=False)

In [174]:
df_kk = DataCleaner('KhonKaen').iloc[5:,:]
df_kk

,datetime,Temp(C),lat,long,WindDir,Wind Speed(km/h),PM2.5
5,2016-03-03 15:00:00,32.100000,16.4467,102.833,85.0,8.000000,35.800000
6,2016-03-03 16:00:00,33.000000,16.4467,102.833,80.0,6.000000,35.800000
7,2016-03-03 17:00:00,31.366667,16.4467,102.833,95.0,6.666667,36.000000
8,2016-03-03 18:00:00,29.733333,16.4467,102.833,110.0,7.333333,35.900000
9,2016-03-03 19:00:00,28.100000,16.4467,102.833,125.0,8.000000,40.166667
...,...,...,...,...,...,...,...
18982,2019-03-18 02:00:00,32.000000,16.4467,102.833,125.0,5.000000,78.200000
18983,2019-03-18 03:00:00,32.000000,16.4467,102.833,125.0,5.000000,80.000000
18984,2019-03-18 04:00:00,32.000000,16.4467,102.833,125.0,5.000000,85.500000
18985,2019-03-18 05:00:00,32.000000,16.4467,102.833,125.0,5.000000,88.300000


In [180]:
df_sk.isna().sum(axis=0)

datetime            0
Temp(C)             0
lat                 0
long                0
WindDir             0
Wind Speed(km/h)    0
PM2.5               0
dtype: int64

In [176]:
df_kk.to_csv('khonkaen.csv', index=False)

In [179]:
df_sk = DataCleaner('Songkhla').iloc[5:,:]
df_sk

,datetime,Temp(C),lat,long,WindDir,Wind Speed(km/h),PM2.5
5,2016-03-03 15:00:00,29.800000,7.1988,100.5951,92.500000,31.500000,31.2
6,2016-03-03 16:00:00,29.300000,7.1988,100.5951,95.000000,31.000000,31.2
7,2016-03-03 17:00:00,28.433333,7.1988,100.5951,91.666667,30.333333,31.2
8,2016-03-03 18:00:00,27.566667,7.1988,100.5951,88.333333,29.666667,31.2
9,2016-03-03 19:00:00,26.700000,7.1988,100.5951,85.000000,29.000000,24.5
...,...,...,...,...,...,...,...
25582,2019-03-18 02:00:00,27.100000,7.1988,100.5951,90.000000,46.000000,28.2
25583,2019-03-18 03:00:00,27.100000,7.1988,100.5951,90.000000,46.000000,30.2
25584,2019-03-18 04:00:00,27.100000,7.1988,100.5951,90.000000,46.000000,30.7
25585,2019-03-18 05:00:00,27.100000,7.1988,100.5951,90.000000,46.000000,31.5


In [181]:
df_sk.to_csv('songkhla.csv', index=False)